# Bank Statement Analysis - Data Cleaning and Preparation

* Importing the necessary libraries.

In [1]:
import pandas as pd 
import numpy as np

* Loading the dataset into a dataframe

In [2]:
df = pd.read_csv("bank_statement.csv")

The data cleaning and preparation process involves:

1. Visual and programmatic assessments to identify any data quality issues
2. Rectifying and cleaning the data to address the identified issues.

### 1.  Assesing the data

* Display the first five rows

In [3]:
df.head()

,Trans. Date,Value. Date,Reference,Debits,Credits,Balance,Originating Branch,Remarks
0,01-Jun-21,01-Jun-21,0NIPT,NaN,"23,125.00","30,543.27",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS\n000014210601170721...
1,01-Jun-21,01-Jun-21,0USAT,500,NaN,"30,043.27",E- CHANNELS,Airtime Purchase USSD-\n101CT00000000026351673...
2,01-Jun-21,01-Jun-21,0NIPU,"1,000.00",NaN,"29,043.27",E- CHANNELS,NIBSS Instant Payment Outward\n000013210601211...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRANSFER BETWEEN CUSTOMERS\n/10.75/REF:0000132...
4,01-Jun-21,01-Jun-21,0NIPU,10,NaN,"29,033.27",E- CHANNELS,COMMISSION 000013210601211604000160255717\nUSS...


* To obtain information about the DataFrame

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396 entries, 0 to 1395
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Trans. Date         1273 non-null   object
 1   Value. Date         1273 non-null   object
 2   Reference           1273 non-null   object
 3   Debits              1108 non-null   object
 4   Credits             165 non-null    object
 5   Balance             1273 non-null   object
 6   Originating Branch  1273 non-null   object
 7   Remarks             1363 non-null   object
dtypes: object(8)
memory usage: 87.4+ KB


### 2.  Rectifying and cleaning the data

Some slight adjustments

* Renaming the columns

In [5]:
for col in df.columns:
    new_col = col.replace(".", "").replace(" ", "_").lower()
    df.rename(columns={col: new_col}, inplace=True)
df.columns

Index(['trans_date', 'value_date', 'reference', 'debits', 'credits', 'balance',
       'originating_branch', 'remarks'],
      dtype='object')

* Dropping the 'value_date' and 'reference' columns from the dataframe using the drop() function. These columns are deemed unnecessary for the analysis, and their removal allows for a more streamlined and efficient analysis.

In [6]:
df.drop(['value_date', 'reference'], axis =1, inplace=True)
df.columns

Index(['trans_date', 'debits', 'credits', 'balance', 'originating_branch',
       'remarks'],
      dtype='object')

* Returning the total number of null values in each column of the dataframe to identify which missing values needS further cleaning and which should be removed

In [7]:
df.isna().sum()

trans_date             123
debits                 288
credits               1231
balance                123
originating_branch     123
remarks                 33
dtype: int64

* Removing rows with null values in the 'trans_date' column, this is necessary to ensure the accuracy and reliability of the remaining data for analysis, as the 'trans_date' column is a crucial attribute for this dataset and cannot have null values

In [8]:
null_rows = np.where(pd.isnull(df.trans_date))[0]

df.drop(labels=null_rows, inplace=True)

df.isna().sum()

trans_date               0
debits                 165
credits               1108
balance                  0
originating_branch       0
remarks                  0
dtype: int64

* Reseting the index of the dataframe after dropping the null rows to ensure that it is ordered sequentially.

In [9]:
df.reset_index(drop=True, inplace=True)

* Replacing the null values in the 'credit' and 'debit' columns with 0 ensures that there are no missing values in the dataset that can potentially affect the data accuracy. This also makes it easier to perform mathematical operations on the data and eliminates the risk of errors or biases in the analysis. 

In [10]:
df.fillna(0, inplace=True)
df.isna().sum()

trans_date            0
debits                0
credits               0
balance               0
originating_branch    0
remarks               0
dtype: int64

* Checking for duplicate values

In [11]:
duplicate = sum(df.duplicated())

print(f'There are {duplicate} duplicate records')

There are 0 duplicate records


* Converting the 'trans_date' column to a datetime format

In [12]:
df['trans_date'] = pd.to_datetime(df['trans_date'])

* Viewing the cleaned data

In [13]:
df.head()

,trans_date,debits,credits,balance,originating_branch,remarks
0,2021-06-01,0,"23,125.00","30,543.27",E- CHANNELS,TRANSFER BETWEEN CUSTOMERS\n000014210601170721...
1,2021-06-01,500,0,"30,043.27",E- CHANNELS,Airtime Purchase USSD-\n101CT00000000026351673...
2,2021-06-01,"1,000.00",0,"29,043.27",E- CHANNELS,NIBSS Instant Payment Outward\n000013210601211...
3,2021-06-01,10,0,"29,033.27",E- CHANNELS,COMMISSION 000013210601211604000160255717\nUSS...
4,2021-06-01,0.75,0,"29,032.52",E- CHANNELS,VALUE ADDED TAX\n00001321060121160400016025571...


* Saving the cleaned DataFrame as a CSV file on my local drive

In [14]:
df.to_csv("statement.csv")